In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from time import time

np.random.seed(1337)

df = pd.read_csv('titanic.csv')

In [2]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S


In [3]:
df_train = df.iloc[:712, :]

df_train = df_train.drop(['Name', 'Ticket', 'Cabin', 'Embarked'], axis=1)

age_mean = df_train['Age'].mean()
df_train['Age'] = df_train['Age'].fillna(age_mean)
df_train['Sex'] = df_train['Sex'].map({'female': 0, 'male': 1}).astype(int)

scaler = StandardScaler()

X_train = scaler.fit_transform(df_train.iloc[:, 2:].values)
y_train = df_train['Survived'].values

In [4]:
df_test = df.iloc[712:, :]

df_test = df_test.drop(['Name', 'Ticket', 'Cabin', 'Embarked'], axis=1)

df_test['Age'] = df_test['Age'].fillna(age_mean)
df_test['Sex'] = df_test['Sex'].map({'female': 0, 'male': 1}).astype(int)

X_test = scaler.transform(df_test.iloc[:, 2:].values)
y_test = df_test['Survived'].values

In [5]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=0, verbose=3)
model = model.fit(X_train, y_train)

y_prediction = model.predict(X_test)
print "\naccuracy", np.sum(y_prediction == y_test) / float(len(y_test))

building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10

accuracy 0.832402234637


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


In [6]:
def softmax(x):
    return np.exp(x) / np.exp(x).sum()

In [7]:
min_loss = 1000000
best_weights = ()

start = time()

for i in xrange(1000):
    W = np.random.rand(2, 6)
    b = np.random.rand(2,)
    loss = 0
    
    for j in xrange(X_train.shape[0]):
        result = np.dot(W, X_train[j]) + b
        result = softmax(result)
        result = -np.log(result)
        label_index = y_train[j]
        loss += result[label_index]
    
    if loss < min_loss:
        print 'loss %s, loop %s' % (round(loss, 3), i)
        min_loss = loss
        best_weights = (W, b)

print '\ntime taken %s seconds' % str(time() - start)

loss 860.362, loop 0
loss 528.333, loop 1
loss 494.827, loop 2
loss 455.383, loop 6
loss 440.769, loop 28
loss 438.998, loop 36
loss 400.598, loop 45
loss 376.708, loop 48
loss 367.815, loop 282
loss 357.662, loop 515

time taken 17.9973759651 seconds


In [8]:
W, b = best_weights
y_prediction = []

for j in xrange(X_test.shape[0]):
    result = np.dot(W, X_test[j]) + b
    result = softmax(result)
    result = np.argmax(result)
    y_prediction.append(result)

y_prediction = np.array(y_prediction)

print "accuracy", np.sum(y_prediction == y_test) / float(len(y_test))

accuracy 0.810055865922


In [9]:
min_loss = 1000000
best_weights = ()

start = time()

for i in xrange(1000):
    W_1 = np.random.rand(100, 6)
    b_1 = np.random.rand(100,)

    W_2 = np.random.rand(2, 100)
    b_2 = np.random.rand(2,)   
    loss = 0

    for j in xrange(X_train.shape[0]):
        result = np.dot(W_1, X_train[j]) + b_1
        result = np.dot(W_2, result) + b_2
        result = softmax(result)
        result = -np.log(result)
        label_index = y_train[j]
        loss += result[y_train[j]]
    
    if loss < min_loss:
        print 'loss %s, loop %s' % (round(loss, 3), i)
        min_loss = loss
        best_weights = (W_1, b_1, W_2, b_2)

print '\ntime taken %s seconds' % str(time() - start)

loss 995.982, loop 0
loss 701.394, loop 2
loss 544.231, loop 6
loss 532.946, loop 29
loss 480.419, loop 42
loss 442.137, loop 51
loss 411.774, loop 123
loss 386.456, loop 384
loss 373.84, loop 709

time taken 16.3448729515 seconds


In [10]:
W_1, b_1, W_2, b_2 = best_weights
y_prediction = []

for j in xrange(X_test.shape[0]):
    result = np.dot(W_1, X_test[j]) + b_1
    result = np.dot(W_2, result) + b_2
    result = softmax(result)
    result = np.argmax(result)
    y_prediction.append(result)

y_prediction = np.array(y_prediction)

print "accuracy", np.sum(y_prediction == y_test) / float(len(y_test))

accuracy 0.804469273743


In [11]:
min_loss = 1000000
best_weights = ()

start = time()

for i in xrange(1000):
    W_1 = np.random.rand(100, 6)
    b_1 = np.random.rand(100,)

    W_2 = np.random.rand(100, 100)
    b_2 = np.random.rand(100,)
    
    W_3 = np.random.rand(2, 100)
    b_3 = np.random.rand(2,)
    loss = 0

    for j in xrange(X_train.shape[0]):
        result = np.dot(W_1, X_train[j]) + b_1
        result = np.dot(W_2, result) + b_2
        result = softmax(result)
        result = -np.log(result)
        label_index = y_train[j]
        loss += result[y_train[j]]
    
    if loss < min_loss:
        print 'loss %s, loop %s' % (round(loss, 3), i)
        min_loss = loss
        best_weights = (W_1, b_1, W_2, b_2, W_3, b_3)

print '\ntime taken %s seconds' % str(time() - start)

loss 7108.058, loop 0
loss 6821.721, loop 4
loss 5176.254, loop 5
loss 3808.894, loop 25
loss 3731.686, loop 321
loss 3208.329, loop 672

time taken 22.4115312099 seconds


In [12]:
W_1, b_1, W_2, b_2, W_3, b_3 = best_weights
y_prediction = []

for j in xrange(X_test.shape[0]):
    result = np.dot(W_1, X_test[j]) + b_1
    result = np.dot(W_2, result) + b_2
    result = np.dot(W_3, result) + b_3    
    result = softmax(result)
    result = np.argmax(result)
    y_prediction.append(result)

y_prediction = np.array(y_prediction)

print "accuracy", np.sum(y_prediction == y_test) / float(len(y_test))

accuracy 0.586592178771
